In [1]:
import Pkg

Pkg.add("HiddenMarkovModels")
Pkg.add("Distributions")
Pkg.add("StableRNGs")
Pkg.add("XLSX")


    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`


In [41]:
using HiddenMarkovModels, Distributions
using LinearAlgebra
using StableRNGs
using XLSX
using DataFrames
using Dates
using Distributions
using Statistics
using CSV

In [3]:
rng = StableRNG(63);

In [4]:
acwi = DataFrame(XLSX.readtable("../esg/acwi_usd_net_03.04.24.xlsx","acwi_usd_net_full", header=true, infer_eltypes=true))
acwi.Date = Date.(acwi.Date, dateformat"u d, Y")
acwi

Row,Date,ACWI
,Date,Float64
1,1988-01-29,0.0249309
2,1988-02-29,0.0576797
3,1988-03-31,0.0307011
4,1988-04-29,0.0125812
5,1988-05-31,-0.0199201
6,1988-06-30,-0.00119118
7,1988-07-29,0.0183463
8,1988-08-31,-0.0551142
9,1988-09-30,0.0428067


In [5]:
describe(acwi.ACWI)

Summary Stats:
Length:         435
Missing Count:  0
Mean:           0.007221
Std. Deviation: 0.043966
Minimum:        -0.198150
1st Quartile:   -0.017986
Median:         0.012573
3rd Quartile:   0.033632
Maximum:        0.123263
Type:           Float64


In [6]:
#= init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); =#

init_guess = [0.5, 0.5]
trans_guess = [0.9 0.1; 0.9 0.1]
#dists_guess = [MvNormal([-0.1, -0.7], I), MvNormal([0.4, 0.7], I)]
dists_guess = [Normal(-0.07,0.20),Normal(0.14,0.07)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [7]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi.ACWI);

In [8]:
last(loglikelihood_evolution)

774.9155810120484

In [9]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.953852   0.0461485
 0.0537106  0.946289

In [10]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.0015860352863334213, σ=0.05502964773221741)
 Normal{Float64}(μ=0.013469406460119963, σ=0.025198679554070277)

In [11]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.019032423436001054
0.190628291589636
regime 2
0.16163287752143957
0.08729078654259356


In [12]:
acwi_short = acwi[Date(2003,01,01) .<= acwi.Date .<= Date(2024,03,31), :]

Row,Date,ACWI
,Date,Float64
1,2003-01-31,-0.0294357
2,2003-02-28,-0.017938
3,2003-03-31,-0.00431422
4,2003-04-30,0.0886378
5,2003-05-30,0.0574971
6,2003-06-30,0.0187239
7,2003-07-31,0.0218749
8,2003-08-29,0.0233834
9,2003-09-30,0.00607447


In [13]:
println(mean(acwi.ACWI))
println(std(acwi.ACWI))
println(mean(acwi_short.ACWI))
println(std(acwi_short.ACWI))


0.007221256079983577
0.04396581164211372
0.00820031798641316
0.044712591384382244


In [14]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi_short.ACWI);

In [15]:
last(loglikelihood_evolution)

459.13892457264535

In [16]:
typeof(hmm_est)

HMM{Vector{Float64}, Matrix{Float64}, Vector{Normal{Float64}}}

In [17]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.932483   0.0675174
 0.0569406  0.943059

In [18]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.001677850881617401, σ=0.05909238647054062)
 Normal{Float64}(μ=0.014143605791660168, σ=0.02345202862629552)

In [19]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.020134210579408812
0.20470203141494414
regime 2
0.169723269499922
0.08124021024260716


In [20]:
initialization(hmm_est)

2-element Vector{Float64}:
 1.0
 5.644524055265908e-50

In [42]:
cpipl = CSV.read("../esg/cpiypl_m.csv", DataFrame)[:,["Data","Zamkniecie"]]

Row,Data,Zamkniecie
,Date,Float64
1,2002-12-31,0.8
2,2003-01-31,0.5
3,2003-02-28,0.5
4,2003-03-31,0.6
5,2003-04-30,0.3
6,2003-05-31,0.4
7,2003-06-30,0.8
8,2003-07-31,0.8
9,2003-08-31,0.7


In [43]:
date_index = Date(2003,01,01) .<= cpipl.Data .<= Date(2024,02,29)
edo = cpipl[date_index,"Zamkniecie"] ./100 .+ 0.0125

254-element Vector{Float64}:
 0.0175
 0.0175
 0.018500000000000003
 0.0155
 0.0165
 0.0205
 0.0205
 0.0195
 0.021500000000000002
 0.025500000000000002
 ⋮
 0.1275
 0.12050000000000001
 0.11349999999999999
 0.09449999999999999
 0.0785
 0.0785
 0.0745
 0.051500000000000004
 0.040499999999999994

In [44]:
acwi_s = acwi[Date(2003,01,01) .<= acwi.Date .<= Date(2024,02,29),"ACWI"]

254-element Vector{Float64}:
 -0.029435692626285204
 -0.01793797150041898
 -0.0043142245259009115
  0.08863778054862848
  0.05749710080604764
  0.018723853620892683
  0.02187491693910637
  0.023383446912552674
  0.006074469437031604
  0.06037793047696027
  ⋮
  0.05806157327945405
  0.03660848514326154
 -0.027944324625502892
 -0.04134978511934306
 -0.030067745405442547
  0.09229752768158961
  0.04803335439585643
  0.0058616798157242744
  0.042913606684614525

In [45]:
returns = hcat(acwi_s,edo)

254×2 Matrix{Float64}:
 -0.0294357   0.0175
 -0.017938    0.0175
 -0.00431422  0.0185
  0.0886378   0.0155
  0.0574971   0.0165
  0.0187239   0.0205
  0.0218749   0.0205
  0.0233834   0.0195
  0.00607447  0.0215
  0.0603779   0.0255
  ⋮           
  0.0580616   0.1275
  0.0366085   0.1205
 -0.0279443   0.1135
 -0.0413498   0.0945
 -0.0300677   0.0785
  0.0922975   0.0785
  0.0480334   0.0745
  0.00586168  0.0515
  0.0429136   0.0405

In [46]:
returns = [returns[i,:] for i in 1:size(returns,1)]

254-element Vector{Vector{Float64}}:
 [-0.029435692626285204, 0.0175]
 [-0.01793797150041898, 0.0175]
 [-0.0043142245259009115, 0.018500000000000003]
 [0.08863778054862848, 0.0155]
 [0.05749710080604764, 0.0165]
 [0.018723853620892683, 0.0205]
 [0.02187491693910637, 0.0205]
 [0.023383446912552674, 0.0195]
 [0.006074469437031604, 0.021500000000000002]
 [0.06037793047696027, 0.025500000000000002]
 ⋮
 [0.05806157327945405, 0.1275]
 [0.03660848514326154, 0.12050000000000001]
 [-0.027944324625502892, 0.11349999999999999]
 [-0.04134978511934306, 0.09449999999999999]
 [-0.030067745405442547, 0.0785]
 [0.09229752768158961, 0.0785]
 [0.04803335439585643, 0.0745]
 [0.0058616798157242744, 0.051500000000000004]
 [0.042913606684614525, 0.040499999999999994]

In [93]:
init_guess = [0.25, 0.25, 0.25, 0.25]
trans_guess = [0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25]
dists_guess = [MvNormal([-0.05, 0.012], I), MvNormal([0.2, 0.037], I), MvNormal([0.07, 0.049], I), MvNormal([-0.2, 0.082], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [94]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns);

In [95]:
last(loglikelihood_evolution)

1086.0423474913564

In [96]:
transition_matrix(hmm_est)

4×4 Matrix{Float64}:
 0.425494     0.234221     0.283825     0.0564599
 0.187719     0.812281     7.55407e-13  3.11054e-20
 0.217072     0.644388     0.13854      5.46727e-28
 4.81994e-21  8.57241e-37  0.0942793    0.905721

In [98]:
obs_distributions(hmm_est)

4-element Vector{IsoNormal}:
 IsoNormal(
dim: 2
μ: [-0.03129398017170018, 0.03477600738241481]
Σ: [0.0005866814416253256 0.0; 0.0 0.0005866814416253256]
)

 IsoNormal(
dim: 2
μ: [0.019660866124438896, 0.032555661006222585]
Σ: [0.00030808785285821175 0.0; 0.0 0.00030808785285821175]
)

 IsoNormal(
dim: 2
μ: [0.07926253831291416, 0.037884960896446176]
Σ: [0.0004508494595335847 0.0; 0.0 0.0004508494595335847]
)

 IsoNormal(
dim: 2
μ: [-0.021833161896736286, 0.1185034538258361]
Σ: [0.003512190550272683 0.0; 0.0 0.003512190550272683]
)


In [52]:
regime1 = obs_distributions(hmm_est)[1]

IsoNormal(
dim: 2
μ: [-0.007180069470621233, 0.07578098201327435]
Σ: [0.0036875320284281313 0.0; 0.0 0.0036875320284281313]
)


In [53]:
mean(regime1) * [12. 1.]

2×2 Matrix{Float64}:
 -0.0861608  -0.00718007
  0.909372    0.075781

In [54]:
cov(regime1)

2×2 Matrix{Float64}:
 0.00368753  0.0
 0.0         0.00368753

In [55]:
cov(regime1)[1,1]*12^0.5

0.01277398565555009

In [56]:
init = [0.6, 0.4]
trans = [0.7 0.3; 0.2 0.8]
dists = [MvNormal([-0.5, -0.8], I), MvNormal([0.5, 0.8], I)]
hmm = HMM(init, trans, dists)

Hidden Markov Model with:
 - initialization: [0.6, 0.4]
 - transition matrix: [0.7 0.3; 0.2 0.8]
 - observation distributions: [IsoNormal(
dim: 2
μ: [-0.5, -0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
, IsoNormal(
dim: 2
μ: [0.5, 0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
]

In [65]:
nb_seqs = 100
long_obs_seqs = [last(rand(rng, hmm, 120)) for k in 1:nb_seqs];
typeof(long_obs_seqs)

Vector{Vector{Vector{Float64}}} (alias for Array{Array{Array{Float64, 1}, 1}, 1})

In [66]:
long_obs_seq_concat = reduce(vcat, long_obs_seqs)
typeof(long_obs_seq_concat)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [67]:
seq_ends = cumsum(length.(long_obs_seqs))
seq_ends'

1×100 adjoint(::Vector{Int64}) with eltype Int64:
 120  240  360  480  600  720  840  960  …  11520  11640  11760  11880  12000

In [68]:
init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [75]:
hmm_est_concat, _ = baum_welch(hmm_guess, long_obs_seq_concat,; seq_ends);

In [76]:
transition_matrix(hmm_est_concat)

2×2 Matrix{Float64}:
 0.686014  0.313986
 0.1935    0.8065

In [77]:
obs_distributions(hmm_est)

2-element Vector{IsoNormal}:
 IsoNormal(
dim: 2
μ: [-0.007180069470621233, 0.07578098201327435]
Σ: [0.0036875320284281313 0.0; 0.0 0.0036875320284281313]
)

 IsoNormal(
dim: 2
μ: [0.014715723918157661, 0.03150717749765352]
Σ: [0.00048574567964616865 0.0; 0.0 0.00048574567964616865]
)


In [60]:
simulated_ACWI = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))[1:1000,:]

1000×50 Matrix{Float64}:
  0.183608   0.040727   0.130638  …   0.310636   0.259417   0.106576
  0.057796  -0.015669   0.01473       0.181475   0.038817   0.09112
 -0.042034   0.071062   0.211914      0.271774   0.03051    0.014144
  0.048233   0.002585   0.069304      0.161997   0.150916  -0.458586
 -0.097277   0.278737   0.249458      0.054621   0.20249    0.148626
  0.168582   0.058298   0.174671  …  -0.057467  -0.037438   0.030723
  0.096997   0.214256  -0.246285      0.148997   0.192833   0.181174
  0.254872   0.054849   0.014113      0.13077    0.066608  -0.076274
  0.263792   0.213822   0.260164      0.309639   0.022386   0.268776
  0.065249  -0.206639   0.347267      0.165357   0.180862   0.298468
  ⋮                               ⋱                        
  0.313924   0.116269   0.06669       0.226258   0.172806   0.186851
  0.296246  -0.045214   0.231189      0.156628   0.218297  -0.019684
 -0.071925  -0.124002  -0.197512      0.287149  -0.01889    0.176335
  0.184442   0.2505